#This project encompasses the LimeLocale's Finance API beta official AI that assists the team in Research in Stock health per company all over the world

#As a starting step, It currently automates the Team's research on Stock and Crypto in China's GBA
#TODO-Measure the accuracy of the model


In [53]:
#Baseline_Setup_and_dependencies
#Pegasus Tokenizer, NLP for summarization
#BS4 scraps the web for news
import sentencepiece
import tensorflow
import requests
import pickle
import re
import tensorflow
from transformers import PegasusTokenizer,PegasusForConditionalGeneration, TFPegasusForConditionalGeneration
from bs4 import BeautifulSoup
from transformers import pipeline


In [54]:
#Loading and saving the summarization models  from the Hugging face model

model_name = "human-centered-summarization/financial-summarization-pegasus"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

'''
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("human-centered-summarization/financial-summarization-pegasus")
model = AutoModelForSeq2SeqLM.from_pretrained("human-centered-summarization/financial-summarization-pegasus")
'''

'\nfrom transformers import AutoTokenizer, AutoModelForSeq2SeqLM\ntokenizer = AutoTokenizer.from_pretrained("human-centered-summarization/financial-summarization-pegasus")\nmodel = AutoModelForSeq2SeqLM.from_pretrained("human-centered-summarization/financial-summarization-pegasus")\n'

This Code slices focus on the Automation of more than one stock symbol

In [55]:
#Building a News and Sentiment Pipleline
#Test with company stock names --Edited to user preference
#The model asks which tinkers you wanna monitor and how much
#You can also edit the search URL to use expedia or Bloomberg finance
#Search for stock news using Google and Yahoo Finance
#TODO--> Figure out how to allow a universal Web scrapping instead of relying solely on Yahoo finance. ie investopedia, Bloomberg LP etc
def search_for_stock_news_urls(ticker):
  search_url = f"https://www.google.com/search?q=yahoo+finance+{ticker}&tbm=nws"
  r = requests.get(search_url)
  soup = BeautifulSoup(r.text, 'html.parser')
  atags = soup.find_all('a')
  hrefs = [link['href'] for link in atags]
  return hrefs

num_tickers = int(input("Enter the number of stock tickers you want to monitor: "))
monitored_tickers = []
for i in range(num_tickers):
  ticker = input(f"Enter the company ticker {i+1}: ")
  monitored_tickers.append(ticker)

raw_urls = {ticker: search_for_stock_news_urls(ticker) for ticker in monitored_tickers}
raw_urls

{'MSFT': ['/?sa=X&ved=0ahUKEwiiwoCa7Jn-AhWJzaQKHfMuBH0QOwgC',
  '/search?q=yahoo+finance+MSFT&tbm=nws&ie=UTF-8&gbv=1&sei=7iIxZKKTEImbkwXz3ZDoBw',
  '/search?q=yahoo+finance+MSFT&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwiiwoCa7Jn-AhWJzaQKHfMuBH0Q_AUIBSgA',
  '/search?q=yahoo+finance+MSFT&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwiiwoCa7Jn-AhWJzaQKHfMuBH0Q_AUIBygC',
  'https://maps.google.com/maps?q=yahoo+finance+MSFT&um=1&ie=UTF-8&sa=X&ved=0ahUKEwiiwoCa7Jn-AhWJzaQKHfMuBH0Q_AUICCgD',
  '/search?q=yahoo+finance+MSFT&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwiiwoCa7Jn-AhWJzaQKHfMuBH0Q_AUICSgE',
  '/search?q=yahoo+finance+MSFT&ie=UTF-8&tbm=shop&source=lnms&sa=X&ved=0ahUKEwiiwoCa7Jn-AhWJzaQKHfMuBH0Q_AUICigF',
  '/search?q=yahoo+finance+MSFT&ie=UTF-8&tbm=bks&source=lnms&sa=X&ved=0ahUKEwiiwoCa7Jn-AhWJzaQKHfMuBH0Q_AUICygG',
  '/advanced_search',
  '/search?q=yahoo+finance+MSFT&ie=UTF-8&tbm=nws&source=lnt&tbs=lr:lang_1zh-CN%7Clang_1zh-TW&lr=lang_zh-CN%7Clang_zh-TW&sa=X&ved=0ahUKEwiiwoCa7Jn-Ah

In [56]:
#Strip out the unwanted URLs--> policies,accounts, preferences etc
excluded_list = ['maps', 'policies', 'preferences', 'support', 'accounts']
def strip_unwanted_urls(urls, excluded_list):
    final_val = []
    for url in urls:
        if 'https://' in url and not any(excluded_word in url for excluded_word in excluded_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            final_val.append(res)
    return list(set(final_val))

cleaned_urls = {ticker:strip_unwanted_urls(raw_urls[ticker], excluded_list) for ticker in monitored_tickers}
cleaned_urls


{'MSFT': ['https://finance.yahoo.com/news/1-u-microsoft-reach-settlement-184020750.html',
  'https://finance.yahoo.com/news/microsoft-escalates-war-against-playstation-214200023.html',
  'https://finance.yahoo.com/news/why-speculative-ai-stocks-like-c3ai-and-bigbearai-are-seeing-monster-volatility-123221071.html',
  'https://finance.yahoo.com/news/microsoft-is-dominating-the-ai-warsfor-now-200037880.html',
  'https://finance.yahoo.com/news/uk-digital-regulator-recommends-competition-152719375.html',
  'https://www.google.com/search?q%3Dyahoo%2Bfinance%2BMSFT%26tbm%3Dnws%26pccc%3D1',
  'https://finance.yahoo.com/news/microsoft-is-bringing-ai-to-its-productivity-apps-in-another-strike-at-google-150058515.html',
  'https://finance.yahoo.com/news/microsoft-abandons-plans-office-london-195627186.html',
  'https://finance.yahoo.com/news/googles-chatgpt-gambit-sundar-pichai-175125233.html',
  'https://finance.yahoo.com/news/microsoft-hr-boss-wrote-legendary-005135292.html',
  'https://finance

In [57]:
#Seach and scrape cleaned URLs
def scrape_and_process(urls):
    ARTICLES = []
    for url in urls:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text = [paragraph.text for paragraph in paragraphs]
        words = ''.join(text).split(' ')[:400]
        ARTICLE = ''.join(words)
        ARTICLES.append(ARTICLE)
    return ARTICLES

articles = {ticker:scrape_and_process(cleaned_urls[ticker]) for ticker in monitored_tickers}
articles

{'MSFT': ['(AddsdetailsofsanctionsviolationsfromTreasurynotice,acknowledgementofviolationsinMicrosoftstatement)ByKanishkaSinghWASHINGTON,April6(Reuters)-TheUnitedStatesandMicrosoftCorpreachedasettlementonThursdayoverthetechfirm\'sapparentviolationsofsanctionsandexportcontrols,whichitdisclosedvoluntarily,thegovernmentandthecompanysaid.Microsofthasagreedtoremitabout$3milliontosettleitspotentialcivilliabilityformorethan1,300apparentsanctionsviolationsinrelationtorestrictionsonCuba,Iran,SyriaandRussiainvolvingtheexportofservicesorsoftwarefromtheU.S.tosanctionedjurisdictions,theU.S.TreasuryDepartmentsaidinastatement.TheTreasuryDepartmentaddedthatMicrosoft\'sconductwas"non-egregiousandvoluntarilyself-disclosed."Amajorityoftheviolations,whichtookplacebetween2012and2019,involvedblockedRussianentitiesorpersonslocatedintheCrimearegionofUkraine,andoccurredasaresultofMicrosoft\'sfailuretoidentifyandpreventtheuseofitsproductsbyprohibitedparties,theTreasuryDepartmentsaid.RussiaannexedCrimeain2014bef

In [59]:
from transformers import pipeline

# Define the summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def summarize_all_articles(articles):
    summaries = []
    for article in articles:
        # Use the summarizer pipeline to generate a summary
        summary = summarizer(article, max_length=120, min_length=30, do_sample=False)[0]['summary_text']
        summaries.append(summary)
    return summaries

summaries = {ticker:summarize_all_articles(articles[ticker]) for ticker in monitored_tickers}


Your max_length is set to 120, but you input_length is only 27. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)


In [60]:
#Calling the sentiment analysis to perform sentiment labelling for the summaries
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model_revision = "af0f99b"
model = AutoModelForSequenceClassification.from_pretrained(model_name, revision=model_revision)
tokenizer = AutoTokenizer.from_pretrained(model_name, revision=model_revision)
sentiment = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

In [61]:
# Adding stop words and phrases
stop_words = ['Ourengineersareworkingquicklytoresolvetheissue. Thankyouforyourpatience. Back to Mail Online home. back to the page you came from.', '2023-私隱權政策-條款. ©2023. www.japanewspaper.com']

# Adding Sentiment Analysis to give a Negative/Positive score
# -VE means the market is not good for the product
# +VE means the market is good for the product/investment
scores = {
    ticker: sentiment(summaries[ticker])
    for ticker in monitored_tickers
}
for ticker in scores:
    print(f"\nSentiment analysis for {ticker}:")
    for i, score in enumerate(scores[ticker]):
        summary = summaries[ticker][i]
        if any(word in summary for word in stop_words) or re.match(r'^\s*$', summary):
            continue  # Skip if the sentiment contains stop words or phrases, or is blank
        print(f"{i+1}. Summary: {summary}")
        print(f"   Score: {score['score']}, Label: {score['label']}")
    negative_count = sum(
        1 for score in scores[ticker] if score['label'] == 'NEGATIVE'
    )
    if negative_count > 5:
        print(f"\nThe model finds that stock {ticker} is not doing well currently. We recommend that you don't buy for short holding.")
    elif negative_count == 5:
        print(f"\nThe model generates a neutral view on stock {ticker}, further research is needed: Bloomberg / Expedia refinement.")
    elif negative_count < 5:
        print(f"\nThe model finds that stock {ticker} is good to buy for the short term. Contact our brokers to buy.")



Sentiment analysis for MSFT:
1. Summary: The United States and MicrosoftCorpreachedasettlementon Thursday. Microsofthasagreedtoremitabout $3milliontosettleitspotentialcivilliabilityformorethan1,300apparentsanctionsviolations. Microsoft's conduct was "non-egregious andvoluntarilyself-disclosed," Treasury said.
   Score: 0.9917056560516357, Label: NEGATIVE
2. Summary: MicrosoftshutdownclassicgameemulationontheXboxSeriesX/Son April6. TheysaidtheynowhadnochoicebuttotagXboxandMicrosoftGamingCEOPhilSpenceron Twitter.
   Score: 0.9874457120895386, Label: NEGATIVE
3. Summary: YahooFinanceiszoominginonshares ofspeculativeandbuzzy AInames. Therearethreethingsinvestors could havebankedonoverthe lastfivetradingsessions.
   Score: 0.9947439432144165, Label: NEGATIVE
4. Summary: Microsoft pouredbillions intoChatGPTdeveloperOpenAI. The techgiant is now adding the technology to everything fromitsDynamics365 toOffice365,and,of course,Bing.
   Score: 0.6947991251945496, Label: POSITIVE
5. Summary: U.K.

This slice is for exporting a summary

In [62]:
#Step-1 is to summarize the blueprint labels for the CSV
#show why the previous tinker is not good to buy
#Fix APPL error
def create_output_array(summaries, scores, urls):
  output = []
  for ticker in monitored_tickers:
    for counter in range(len(summaries[ticker])):
      output_this = [
      ticker,
      summaries[ticker][counter],
      scores[ticker][counter],
      urls[ticker][counter],
    ]
    output.append(output_this)
  return output

final_output = create_output_array(summaries,scores,cleaned_urls)
final_output

[['MSFT',
  "Googlesaidthesystemsarefasterandmorepower-efficientthancomparablesystems from\xa0N NvidiaCorp. The company's\xa0Bard and MicrosoftCorp'sbackedChatGPT-WhatItTakesToTrainAI.",
  {'label': 'NEGATIVE', 'score': 0.952090859413147},
  'https://finance.yahoo.com/news/googles-bard-vs-microsoft-backed-140424897.html']]

In [63]:
#Exporting the results
#TOCO--> Fix the csv output
final_output.insert(0, ['Ticker', 'Summary', 'Label', 'Confidence', 'Url'])
import csv
with open('assetsummaries.csv', mode='w', newline='') as f:
  csv_writer = csv.writer(f, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
  csv_writer.writerows(final_output)